In [24]:
import wandb
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
wandb.init(project="haselab-conpetition", name=f"experiment-{timestamp}")

In [16]:
import zipfile
from pathlib import Path
from PIL import Image
import subprocess

import torch
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

In [17]:
# データセットをダウンロード・展開するコード
# 学外にいる場合や，Google Colaboratory で実行する場合，このコードはうまく動かない．
# データセットをアクセスできる場所に置き，dataset_dir にデータセットのパスを指定する必要がある．

# 具体的手順は以下
# Google Colaboratory で実行する場合
# 1. Aの部分をコメントアウト
# 2. zipファイルをアップロードする．
# 3. Bの部分をアンコメント

# それ以外
# 1. competition_images/ をアクセスできる場所に置く
# 2. Aの部分をコメントアウト
# 3. Cをアンコメントし，competitions_images/ のパスとなる文字列を格納  Ex.) dataset_dir = "/home/haselab/Documents/tat/tmp/competition_images"



#      ----- A -----
#      # データセットの保存先を指定
#      save_dir = str(Path().resolve()) # 保存ディレクトリをノートブックと同じディレクトリに設定
#      dataset_dir = save_dir + "/competition_images/"
#      # データセットを保存
#      if Path(dataset_dir).exists():
#          print(f"Dataset directory already exists: {dataset_dir}")
#      else:
#          zip_path = Path(str(save_dir)) / "tmp.zip"
#          url = "http://10.0.87.42:8080/dataset" # こっちでもいける
#          subprocess.run(["wget", url, "-O", zip_path, "--no-proxy", "-q"], check=True) # .pyならこっち
#          # !wget -O {zip_path} {url} -q --no-proxy
#          with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#              zip_ref.extractall(save_dir)
#              if zip_path.exists():
#                  zip_path.unlink()
#          print(f"Downloaded to {save_dir} and extracted.")
#      ----- A -----



# ----- B -----
# zip_path = "/content/competition_images.zip" # zipファイルのパスを指定
# ext_dir = "/content/"  # 展開先を指定

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(ext_dir)
# dataset_dir = ext_dir + "competition_images/" # データセットのパスを指定
# ----- B -----


# ----- C -----
dataset_dir = "./content/competition_images" # 要変更．データセットのパスを指定
# ----- C -----


In [18]:
# テストデータ読み込みのためのカスタムデータセットクラス
class TestDataset(Dataset):
    def __init__(self, path, transform=None, target_transform=None):
        self.img_paths = sorted([p for p in Path(path).iterdir()])
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        path = self.img_paths[index]
        data = Image.open(path).convert('RGB')

        if self.transform: # 画像の前処理
            data = self.transform(data)
        if self.target_transform:
            path = self.target_transform(path) # 画像のパスに前処理している。名前を変更？
        return data, str(path.name)

    def __len__(self):
        return len(self.img_paths)

In [25]:
# ハイパーパラメータを指定
epochs = 5
lr = 0.001
batch_size = 128

wandb.config.update({
    "epochs": epochs,
    "learning_rate": lr,
    "batch_size": batch_size
})

In [20]:
# モデルと訓練に必要なものを定義
num_classes = 4

train_tf = torchvision.transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3, 0.3, 0.3], std=[0.3, 0.3, 0.3])
])
test_tf = torchvision.transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3, 0.3, 0.3], std=[0.3, 0.3, 0.3])
])

train_ds = ImageFolder(root=dataset_dir + "/train_val", transform=train_tf)
test_ds = TestDataset(path=dataset_dir + "/test", transform=test_tf)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

model = torchvision.models.resnet18(num_classes=num_classes) # 事前学習済みモデルを使用しない重みはランダム初期化

# model = models.resnet18(pretrained=True) # 事前学習済みモデルを使用する。ファインチューニング
#num_ftrs = model.fc.in_features # 元の全結合層の入力特徴数を入手
#model.fc = nn.Linear(num_ftrs, 10) # 全結合層を新しく定義

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device : " + ( "GPU" if device.type == "cuda" else "CPU" )) #どっちつかってるか確認

using device : GPU


In [21]:
# 訓練フローを定義
def train_1epoch(model, train_dl, criterion, optimizer, scheduler=None, device="cpu"):
    total_loss = 0.0
    total_corr = 0

    model.train()                                       # 訓練モードに設定
    for inputs, labels in train_dl:                     # ミニバッチを取得してループ
        inputs = inputs.to(device)                      # GPU に転送
        labels = labels.to(device)                      # GPU に転送

        outputs = model(inputs)                         # 順伝播

        loss = criterion(outputs, labels)               # 損失計算
        preds = torch.argmax(outputs.detach(), dim=1)   # 予測値を取得 各データの予測の最大値のインデックスを収集する操作なので、別に追跡する必要はないってかしたら勾配計算が異常になるのでdetach()。
        corr = torch.sum(preds == labels.data).item()   # 正解数をカウント

        optimizer.zero_grad()                           # 勾配を初期化
        loss.backward()                                 # 誤差逆伝播
        optimizer.step()                                # 重み更新

        total_loss += loss.item() * len(inputs)         # 損失を累積
        total_corr += corr                              # 正解数を累積

    if scheduler is not None:                           # 学習率スケジューラが指定されている場合
        scheduler.step()                                # 学習率を更新
    train_loss = total_loss / len(train_dl.dataset)     # 平均損失を計算
    train_acc = total_corr / len(train_dl.dataset)      # 平均精度を計算

    return train_loss, train_acc

In [22]:
# 推論フローを定義
def pred(model, test_dl, device, probs=False, categorize=False):
    total_outputs = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for inputs, labels in test_dl:
            inputs = inputs.to(device)

            outputs = model(inputs)

            if probs:
                outputs = torch.softmax(outputs, dim=1)

            if categorize:
                outputs = torch.argmax(outputs, dim=1)

            total_outputs.append(outputs)
            all_labels.extend(labels)

    outputs = torch.cat(total_outputs, dim=0).cpu().tolist()
    return outputs, all_labels

In [26]:
# 訓練
model = model.to(device)
for epoch in range(epochs):
    train_loss, train_acc = train_1epoch(model, train_dl, criterion, optimizer, device=device)
    print(f"epoch {epoch+1:>3}/{epochs:>3}: train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}")

    wandb.log({"train_accuracy": train_acc, "train_loss": train_loss}) # wandb にログを記録

wandb.save(f"model_epoch{epoch+1}.pth") # wandb にモデルを保存
wandb.finish() # wandb のセッションを終了

epoch   1/  5: train_loss: 0.3398, train_acc: 0.8775
epoch   2/  5: train_loss: 0.2880, train_acc: 0.8917
epoch   3/  5: train_loss: 0.3109, train_acc: 0.8900
epoch   4/  5: train_loss: 0.2646, train_acc: 0.9050
epoch   5/  5: train_loss: 0.2555, train_acc: 0.9042


train_accuracy,▁▅▄██
train_loss,█▄▆▂▁
train_accuracy,0.90417
train_loss,0.25546


In [ ]:
# 推論・結果をCSV形式で保存
csv_name = "predictions.csv"

outputs, all_labels = pred(model, test_dl, device, categorize=True)
lines = [f"{l},{o}" for o, l in zip(outputs, all_labels)]
csv_text = "\n".join(lines)

with open(csv_name, "w", encoding="utf-8") as f:
    f.write(csv_text)